# 语义搜索
1. 读取文档， 按照页来管理，Document（文档的页）,List[Dpocument]
2. 分割文档，按照文本段落来管理，Chunck（段落）,List[Chunck]
3. 向量化： 文本段向量映射，需要嵌入模型来辅助
4. 向量库：把多个文本段存储到向量库


In [1]:
from langchain_community.document_loaders import PyPDFLoader
from pathlib import Path

from langchain_text_splitters import RecursiveCharacterTextSplitter

PDF_FILE_PATH = "../data/中华人民共和国刑法.pdf"

loader = PyPDFLoader(file_path=Path(PDF_FILE_PATH))
docs = loader.load()
# print(len(docs), docs[0].__dict__)


# 文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, add_start_index=True, chunk_overlap=200
)

all_splits = text_splitter.split_documents(docs)
# print(len(all_splits), all_splits[0].__dict__)

# 向量化：嵌入模型辅助
from langchain_ollama.embeddings import OllamaEmbeddings

model_name = "qwen3-embedding:4b"
embedder = OllamaEmbeddings(model=model_name)
# vector_0 = embedder.embed_query(text=all_splits[0].page_content)
# print(len(vector_0), vector_0)

# 向量数据库存储
from langchain_chroma import Chroma

vector_db = Chroma(
    collection_name="china_law",
    persist_directory="../data/chroma_db",
    embedding_function=embedder,
)

ids = vector_db.add_documents(documents=all_splits)
print(len(ids), type(ids))


153 <class 'list'>


In [2]:
# 相似度查询
from langchain_chroma import Chroma

vector_db = Chroma(
    collection_name="china_law",
    persist_directory="../data/chroma_db",
    embedding_function=embedder,
)
user_input = "刑法总共有多少法条？"
# rets = vector_db.similarity_search(query=user_input)
# print(len(rets), rets[0].__dict__)
# for ret in rets:
#     print(ret.page_content)

# 带分数的相似度查询
results = vector_db.similarity_search_with_score(query=user_input)
print(len(results), results[0])


4 (Document(id='ff8c9a7c-03cd-4ed0-8cf5-36e9916f4567', metadata={'author': 'YF-INT6', 'moddate': '2021-06-08T11:30:48+08:00', 'source': '..\\data\\中华人民共和国刑法.pdf', 'creator': 'Microsoft® Word 2019', 'start_index': 0, 'page': 6, 'creationdate': '2021-06-08T11:30:48+08:00', 'page_label': '7', 'producer': 'Microsoft® Word 2019', 'total_pages': 153}, page_content='－7－  \n在外国已经受过刑罚处罚的，可以免除或者减轻处罚。 \n第十一条 享有外交特权和豁免权的外国人的刑事责任，通\n过外交途径解决。 \n第十二条 中华人民共和国成立以后本法施行以前的行为，\n如果当时的法律不认为是犯罪的，适用当时的法律；如果当时的\n法律认为是犯罪的，依照本法总则第四章第八节的规定应当追诉\n的，按照当时的法律追究刑事责任，但是如果本法不认为是犯罪\n或者处刑较轻的，适用本法。 \n本法施行以前，依照当时的法律已经作出的生效判决，继续\n有效。 \n \n第二章 犯罪 \n \n第一节 犯罪和刑事责任 \n \n第十三条 一切危害国家主权、 领土完整和安全， 分裂国家、\n颠覆人民民主专政的政权和推翻社会主义制度，破坏社会秩序和\n经济秩序，侵犯国有财产或者劳动群众集体所有的财产，侵犯公\n民私人所有的财产， 侵犯公民的人身权利、 民主权利和其他权利，\n以及其他危害社会的行为，依照法律应当受刑罚处罚的，都是犯\n罪，但是情节显著轻微危害不大的，不认为是犯罪。 \n第十四条 明知自己的行为会发生危害社会的结果，并且希'), 0.7600553631782532)


In [4]:
# 使用向量查询
from langchain_ollama.embeddings import OllamaEmbeddings

model_name = "qwen3-embedding:4b"
embedder = OllamaEmbeddings(model=model_name)
user_input = "刑法总共有多少法条？"
query_embedding = embedder.embed_query(user_input)
results = vector_db.similarity_search_by_vector(query_embedding)
print(len(results), results[0].__dict__)


4 {'id': 'ff8c9a7c-03cd-4ed0-8cf5-36e9916f4567', 'metadata': {'source': '..\\data\\中华人民共和国刑法.pdf', 'moddate': '2021-06-08T11:30:48+08:00', 'total_pages': 153, 'page': 6, 'start_index': 0, 'creationdate': '2021-06-08T11:30:48+08:00', 'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'page_label': '7', 'author': 'YF-INT6'}, 'page_content': '－7－  \n在外国已经受过刑罚处罚的，可以免除或者减轻处罚。 \n第十一条 享有外交特权和豁免权的外国人的刑事责任，通\n过外交途径解决。 \n第十二条 中华人民共和国成立以后本法施行以前的行为，\n如果当时的法律不认为是犯罪的，适用当时的法律；如果当时的\n法律认为是犯罪的，依照本法总则第四章第八节的规定应当追诉\n的，按照当时的法律追究刑事责任，但是如果本法不认为是犯罪\n或者处刑较轻的，适用本法。 \n本法施行以前，依照当时的法律已经作出的生效判决，继续\n有效。 \n \n第二章 犯罪 \n \n第一节 犯罪和刑事责任 \n \n第十三条 一切危害国家主权、 领土完整和安全， 分裂国家、\n颠覆人民民主专政的政权和推翻社会主义制度，破坏社会秩序和\n经济秩序，侵犯国有财产或者劳动群众集体所有的财产，侵犯公\n民私人所有的财产， 侵犯公民的人身权利、 民主权利和其他权利，\n以及其他危害社会的行为，依照法律应当受刑罚处罚的，都是犯\n罪，但是情节显著轻微危害不大的，不认为是犯罪。 \n第十四条 明知自己的行为会发生危害社会的结果，并且希', 'type': 'Document'}


In [5]:
# 语义搜索封装成chain

from langchain_core.runnables import chain


@chain
def retriever(query: str):
    return vector_db.similarity_search(query)


rets = retriever.invoke("如何使用langchain")
print(len(rets), rets[0])


4 page_content='－102－ 
人员和其他直接责任人员，依照前款的规定处罚。 
有前两款行为，同时构成其他犯罪的，依照处罚较重的规定
定罪处罚。 
第二百八十七条 利用计算机实施金融诈骗、盗窃、贪污、
挪用公款、窃取国家秘密或者其他犯罪的，依照本法有关规定定
罪处罚。 
第二百八十七条之一 利用信息网络实施下列行为之一，情
节严重的，处三年以下有期徒刑或者拘役，并处或者单处罚金： 
（一）设立用于实施诈骗、传授犯罪方法、制作或者销售违
禁物品、管制物品等违法犯罪活动的网站、通讯群组的； 
（二）发布有关制作或者销售毒品、枪支、淫秽物品等违禁
物品、管制物品或者其他违法犯罪信息的； 
（三）为实施诈骗等违法犯罪活动发布信息的。 
单位犯前款罪的，对单位判处罚金，并对其直接负责的主管
人员和其他直接责任人员，依照第一款的规定处罚。 
有前两款行为，同时构成其他犯罪的，依照处罚较重的规定
定罪处罚。 
第二百八十七条之二 明知他人利用信息网络实施犯罪，为
其犯罪提供互联网接入、服务器托管、网络存储、通讯传输等技
术支持，或者提供广告推广、支付结算等帮助，情节严重的，处
三年以下有期徒刑或者拘役，并处或者单处罚金。 
单位犯前款罪的，对单位判处罚金，并对其直接负责的主管' metadata={'page_label': '102', 'creator': 'Microsoft® Word 2019', 'author': 'YF-INT6', 'moddate': '2021-06-08T11:30:48+08:00', 'creationdate': '2021-06-08T11:30:48+08:00', 'page': 101, 'start_index': 0, 'producer': 'Microsoft® Word 2019', 'total_pages': 153, 'source': '..\\data\\中华人民共和国刑法.pdf'}
